In [9]:

import requests 
from bs4 import BeautifulSoup
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pandas as pd
import numpy as np
from datetime import datetime
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ChromeOptions
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
options=ChromeOptions()
options.headless= False
driver = webdriver.Chrome(options=options)
name_of_coin=input("Please Enter the Name of COIN IN LOWERCASE: ")
url = f"https://coinmarketcap.com/currencies/{name_of_coin}/historical-data/"
driver.get(url)
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "table")))

currancy_details=[]

# Extract data form historical table
def extract_data():
    soup=BeautifulSoup(driver.page_source,'lxml')
    table=soup.find("table")
    if table:
        tbody = table.find('tbody')
        rows = tbody.find_all('tr')
        
        for row in rows:
            cols = row.find_all('td')
            cols = [ele.text.strip() for ele in cols]
            currancy_details.append({
                        "Date":cols[0], 
                        "Open($)":cols[1], 
                        "High($)":cols[2],  
                        "Low($)":cols[3], 
                        "Close($)":cols[4],  
                        "Volume($)":cols[5], 
                        "Market Cap($)":cols[6]  
                    })
def save_to_csv():
    if not currancy_details:
        print("No data available to save.")
        return
    
    csv_filename = f"{name_of_coin}_historical_data.csv"
    keys = ["Date", "Open($)", "High($)", "Low($)", "Close($)", "Volume($)", "Market Cap($)"]

    with open(csv_filename, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.DictWriter(file, fieldnames=keys)
        writer.writeheader()  
        writer.writerows(currancy_details)

    print(f"Data successfully saved to {csv_filename}")
# click on load more to have the historical data for the coin 
def press_load_more():
    while True:
        try:
            load_more = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Load More')]"))
            )
            load_more.click()
            time.sleep(2)  
            extract_data()  
        except Exception as e:
            print("No more 'Load More' button found or an error occurred. Stopping loop.")
            print({e})
            break
    save_to_csv()

extract_data()
press_load_more()
save_to_csv()
driver.quit()



No more 'Load More' button found or an error occurred. Stopping loop.
{TimeoutException()}
Data successfully saved to solana_historical_data.csv
Data successfully saved to solana_historical_data.csv


In [15]:
def clean_data(csv_filename):
    cleaned_data = []
    
    with open(csv_filename, mode="r", encoding="utf-8") as file:
        reader = csv.reader(file)
        headers = next(reader)
        cleaned_data.append(headers) 
        for row in reader:
            
            row[0] = datetime.strptime(row[0], "%b %d, %Y").strftime("%Y-%m-%d")

           
            for i in range(1, len(row)):
                row[i] = row[i].replace("$", "").replace(",", "")  
                try:
                    row[i] = float(row[i]) 
                except ValueError:
                    pass  

            cleaned_data.append(row)

    return cleaned_data
def save_cleaned(cleaned_data, output_filename):
    """
    Saves cleaned data to a CSV file.
    """
    if not cleaned_data:
        print("No data available to save.")
        return
    
    with open(output_filename, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerows(cleaned_data)

    print(f"Cleaned data successfully saved to {output_filename}")
csv_filename = f"{name_of_coin}_historical_data.csv"
cleaned_data = clean_data(csv_filename)
output_filename = f"{name_of_coin}_historical_data_cleaned.csv"
save_cleaned(cleaned_data, output_filename)

Cleaned data successfully saved to solana_historical_data_cleaned.csv


In [17]:
options=ChromeOptions()
options.headless= False
driver_2 = webdriver.Chrome(options=options)
url_2="https://coinmarketcap.com/"
driver_2.get(url_2)
WebDriverWait(driver_2, 10).until(EC.presence_of_element_located((By.TAG_NAME, "table")))
top_10=[]
names=[]
# Extract data from top coins table
def top_10_coins():
    soup=BeautifulSoup(driver_2.page_source,'lxml')
    table_topCoin=soup.find("table")
    if table_topCoin:
            tbody = table_topCoin.find('tbody')
            rows = tbody.find_all('tr')
            
            for row in rows[:10]:
                cols = row.find_all('td')
                cols[2]=cols[2].find('p')
                cols = [ele.text.strip().lower() for ele in cols]
                top_10.append(cols[2:10])
                names.append(cols[2])
    csv_filename = "top_10_coins.csv"
    with open(csv_filename, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow(["Name", "Price", "1h %", "24h %", "7d %", "Market Cap", "Volume(24h)", "Circulating Supply"])
        
        writer.writerows(top_10)

    print(f"Top 10 coins data saved to {csv_filename}")

    return csv_filename ,names
                
top_10_coins()

driver_2.quit()

Top 10 coins data saved to top_10_coins.csv
